In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def plot_accuracy_loss(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

# Set hyperparameters
batch_size = 64
PATH = "../data/CUB_200_2011"
labels = pd.read_csv(os.path.join(PATH, "image_class_labels.txt"), sep=" ", header=None, names=["img_id", "label"])
images = pd.read_csv(os.path.join(PATH, "images.txt"), sep=" ", header=None, names=["img_id", "filepath"])
classes = pd.read_csv(os.path.join(PATH, "classes.txt"), sep=" ", header=None, names=["label", "category"])
num_classes = len(classes)
df = pd.merge(images, labels, on="img_id")
df = pd.merge(df, classes, on="label")
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# Initialize transformations for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

# Load the dataset and create generators
train_generator = train_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    dataframe=train_df,
    x_col="filepath",
    y_col="category",
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

test_generator = test_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=test_df,
    x_col="filepath",
    y_col="category",
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

val_generator = val_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=val_df,
    x_col="filepath",
    y_col="category",
)




2023-12-18 01:38:05.214406: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 01:38:05.214447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 01:38:05.215013: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 01:38:05.218403: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 01:38:05.771766: W tensorflow/compiler/tf2

Found 9430 validated image filenames belonging to 200 classes.
Found 1179 validated image filenames belonging to 200 classes.
Found 1179 validated image filenames belonging to 200 classes.


In [2]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
num_epochs = 200
leanring_rate = 0.0001

vgg16Model = VGG16(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(vgg16Model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=vgg16Model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=leanring_rate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/vgg16_EARLY.h5')
model = tf.keras.models.load_model('../models/vgg16_EARLY.h5')
model.evaluate(test_generator)

2023-12-18 01:38:06.556420: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 01:38:06.569830: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 01:38:06.569893: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 01:38:06.571462: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 01:38:06.571504: I external/local_xla/xla/stream_executor

Epoch 1/200


2023-12-18 01:38:09.391943: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-18 01:38:11.569517: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8dddba34b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-18 01:38:11.569548: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-12-18 01:38:11.573032: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702881491.634650   25140 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-12-18 01:38:25.426737: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.46GiB with freed_by_count=0. The caller indicates 

148/148 [==============================] - 110s 588ms/step - loss: 5.3498 - accuracy: 0.0031 - val_loss: 5.2980 - val_accuracy: 0.0051
Epoch 2/200
148/148 [==============================] - ETA: 0s - loss: 5.2990 - accuracy: 0.0031

In [ ]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
num_epochs = 200
learning_rate = 0.0001
# Define the model
learning_rate = 0.0001
# Define the model
resnetModel = ResNet50(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(resnetModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=resnetModel.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/ResNet50_EARLY.h5')
model = tf.keras.models.load_model('model_EARLY.h5')
model.evaluate(test_generator)

In [ ]:
num_epochs = 200
leanring_rate = 0.0001

vgg16Model = VGG16(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(vgg16Model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=vgg16Model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=leanring_rate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/vgg16.h5')
model = tf.keras.models.load_model('../models/vgg16.h5')
model.evaluate(test_generator)

In [ ]:
num_epochs = 200
learning_rate = 0.0001
# Define the model
resnetModel = ResNet50(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(resnetModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=resnetModel.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/ResNet50.h5')
model = tf.keras.models.load_model('model.h5')
model.evaluate(test_generator)

#Inception with 229*299 images

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

# Load the dataset and create generators
train_generator = train_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    dataframe=train_df,
    x_col="filepath",
    y_col="category",
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

test_generator = test_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=test_df,
    x_col="filepath",
    y_col="category",
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

val_generator = val_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=val_df,
    x_col="filepath",
    y_col="category",
)

In [ ]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
num_epochs = 200
learning_rate = 0.0001
# Define the model
inceptionModel_EARLY = InceptionV3(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(inceptionModel_EARLY.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inceptionModel_EARLY.input, outputs=predictions)
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs,validation_data=val_generator)
plot_accuracy_loss(history)
model.save('../models/InceptionV3_EARLY.h5')

model = tf.keras.models.load_model('../models/InceptionV3_EARLY.h5')
model.evaluate(test_generator)

In [ ]:
num_epochs = 200
learning_rate = 0.0001
# Define the model
inceptionModel = InceptionV3(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(inceptionModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inceptionModel.input, outputs=predictions)
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs,validation_data=val_generator)
plot_accuracy_loss(history)
model.save('../models/InceptionV3.h5')

model = tf.keras.models.load_model('../models/InceptionV3.h5')
model.evaluate(test_generator)